# Pandas Tip: 
# - pd.merge(): join tables by key
# - pd.melt(): turn columns into rows
# - pd.pivot(): turn rows into columns


In [7]:
import pandas as pd
import numpy as np
import psycopg2 as ps
import matplotlib.pyplot as plt

# Connect to PostgreSQL (optional: comment out if not running locally)
# Make sure PostgreSQL is running and the 'Project' database exists
# conn = ps.connect(
#     host='localhost',
#     database='Project',
#     user='your_username',
#     password='your_password',
#     port=5432
# )


In [91]:
def columns_treatment(df):
    # Standardize column names for consistency
    df.columns = [x.upper() for x in df.columns]
    df.columns = [x.replace(' ', '_') for x in df.columns]
    df.columns = [x.replace('/', '_') for x in df.columns]
    df.columns = [x.replace(',', '_') for x in df.columns]
    df.columns = [x.replace('-', '_') for x in df.columns]
    df.columns = [x.replace("'", '') for x in df.columns]
    return df

def insert_into_table(conn, df, tablename):
    # Insert DataFrame records into a SQL table
    curr = conn.cursor()
    try:
        
        command = f"""INSERT INTO {tablename} (""" 
        final_part =''

        for idx in df.columns:
            command = command + idx + ', '
            final_part = final_part +'%s,'

        command = command[:-2] + ') VALUES(' + final_part[:-1] + ')'

        insert_into_videos = (command)


        for i, row in df.iterrows():
            row_to_insert = []

            for column in df.columns:
                row_to_insert = row_to_insert + [row[column]]

            curr.execute(insert_into_videos, row_to_insert)
            
    except Exception as e:
        print(f'FALHA NO PREENCHIMENTO DA TABELA: {e}')
        curr.execute("ROLLBACK")
    conn.commit()
    
    
def create_table(df, tablename,  conn):
    # Create SQL table from DataFrame and insert records

    command = f"""CREATE TABLE {tablename} ( """

    for idx in df.columns:
        command = command+idx+' '
        if pd.api.types.is_integer_dtype(df[idx]):
            command = command+'INT,'
        elif pd.api.types.is_float_dtype(df[idx]):
            command = command+'FLOAT,'
        else:
            command = command+'VARCHAR,'
    command = command[:-1]        
    command = command + ' )'
    
    
    curr = conn.cursor()
    try:
        curr.execute(f'DROP TABLE IF EXISTS {tablename}')
        create_table_command = (command)

        curr.execute(create_table_command)
    except Exception as e:
        print(f'FALHA NA CRIAÇÃO DA TABELA: {e}')
        curr.execute("ROLLBACK")
    conn.commit()
    
    insert_into_table(conn, df,tablename)
    
    

def mg(query,conn,df):
    # Merge SQL query result into a DataFrame by common keys
    
    #query = "SELECT * FROM ADULT_MORTALITY;"
    df1 = pd.read_sql(query, conn)
    
    try:
        df1=df1[df1['sex']=='Both sexes']
        df1=df1.drop('sex',axis=1)
    except: pass
        
    
    try:
        df_final = df.merge(df1, left_on=['country_name','country_code','year'], right_on=['country_name','country_code','year'],how='left')
    except:
        df_final = df.merge(df1, left_on=['country_name','year'], right_on=['country_name','year'],how='left')

    try:
        df_final.drop(['region_code', 'region'],axis=1,inplace=1)
    except:
        pass
    
    return df_final

# CSV-based merge function (no SQL required)
def mg_csv(df_to_merge, df_base):
    try:
        df_to_merge = df_to_merge[df_to_merge['SEX'] == 'Both sexes']
        df_to_merge = df_to_merge.drop('SEX', axis=1)
    except:
        pass

    try:
        df_final = df_base.merge(df_to_merge,
                                 on=['COUNTRY_NAME', 'COUNTRY_CODE', 'YEAR'],
                                 how='left')
    except:
        df_final = df_base.merge(df_to_merge,
                                 on=['COUNTRY_NAME', 'YEAR'],
                                 how='left')
    return df_final


In [18]:
# Load and reshape life expectancy data
df = pd.read_csv('../data/raw/Life_expectancy_at_birth_total_years.csv')

# Drop metadata columns and unpivot year columns
df = pd.melt(df.drop(['Indicator Code', 'Indicator Name'], axis=1),
             id_vars=['Country Name', 'Country Code'],
             var_name='YEAR', value_name='LIFE_EXPECTANCY')

# Standardize column names
df = columns_treatment(df)

# Remove missing values and convert YEAR to int
df.dropna(inplace=True)
df = df.astype({"YEAR": int})

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='LIFE_EXPECTANCY', conn=conn)


df.to_csv('../data/processed/Life_expectancy.csv', index=False)

COUNTRY_NAME        object
COUNTRY_CODE        object
YEAR                 int32
LIFE_EXPECTANCY    float64
dtype: object


In [19]:
# Load and reshape GDP data
df = pd.read_csv('../data/raw/GDP.csv')

# Drop metadata columns and unpivot year columns
df = pd.melt(df.drop(['Indicator Code', 'Indicator Name'], axis=1),
             id_vars=['Country Name', 'Country Code'],
             var_name='YEAR', value_name='GDP per capita')

# Clean column names, drop missing, convert year
df = columns_treatment(df)
df.dropna(inplace=True)
df = df.astype({"YEAR": int})

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='GDP', conn=conn)

df.head()

df.to_csv('../data/processed/GDP.csv', index=False)

COUNTRY_NAME       object
COUNTRY_CODE       object
YEAR                int32
GDP_PER_CAPITA    float64
dtype: object


In [20]:
# Load and clean overweight children data
df = pd.read_csv('../data/raw/Children-who-are-overweight-sdgs.csv')

# Rename key columns for clarity
df.rename(columns={
    'Indicator 2.2.2b: Prevalence of overweight in children aged 2-4 (%) - Past - Unscaled':
    'overweight in children aged 2 to 4 percentage',
    'Entity': 'Country_name',
    'Code': 'Country_code'
}, inplace=True)

# Drop missing values and clean column names
df.dropna(inplace=True)
df = columns_treatment(df)

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='OVERWEIGHT_IN_CHILDREN', conn=conn)

df.head()


df.to_csv('../data/processed/overweight_in_children.csv', index=False)

COUNTRY_NAME                                      object
COUNTRY_CODE                                      object
YEAR                                               int64
OVERWEIGHT_IN_CHILDREN_AGED_2_TO_4_PERCENTAGE    float64
dtype: object


In [88]:
# Load and clean adult mortality rate data
df = pd.read_csv('../data/raw/Adult_mortality_rate.csv')

# Keep relevant columns only
df = df[['Location', 'SpatialDimValueCode', 'Period', 'Value']]

# Rename for consistency
df.rename(columns={
    'Location': 'country name',
    'SpatialDimValueCode': 'country code',
    'Period': 'year',
    'Value': 'Adult mortality rate probability of dying between 15 and 60 years per 1000 population'
}, inplace=True)

# Drop missing values and clean column names
df.dropna(inplace=True)
df = columns_treatment(df)

# Check data types
print(df.dtypes)

# Save to CSV
df.to_csv('../data/processed/Adult_mortality_rate.csv', index=False)

# Save to SQL (optional, if using a database)
# create_table(df=df, tablename='ADULT_MORTALITY', conn=conn)

# Optional preview
df[df['COUNTRY_NAME'] == 'Brazil'].head()


COUNTRY_NAME                                                                              object
COUNTRY_CODE                                                                              object
YEAR                                                                                       int64
ADULT_MORTALITY_RATE_PROBABILITY_OF_DYING_BETWEEN_15_AND_60_YEARS_PER_1000_POPULATION    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,ADULT_MORTALITY_RATE_PROBABILITY_OF_DYING_BETWEEN_15_AND_60_YEARS_PER_1000_POPULATION
89,Brazil,BRA,2016,142.80
179,Brazil,BRA,2016,193.50
518,Brazil,BRA,2016,91.38
642,Brazil,BRA,2015,144.50
735,Brazil,BRA,2015,195.70


In [21]:
# Load and prepare daily caloric supply data
df = pd.read_csv('../data/raw/Daily supply of calories _ adult-men-overweight-or-obese-vs-daily-supply-of-calories.csv')

# Drop unused columns and rename for consistency
df.drop(['Population (historical estimates)', 'Continent'], axis=1, inplace=True)

df.rename(columns={
    'Entity': 'Country name',
    'Code': 'Country Code',
    'Daily caloric supply (FAO (2017) & Various historical sources)': 'Daily caloric supply',
    'Overweight or Obese (NCDRisC (2017))': 'Overweight or Obese'
}, inplace=True)

# Drop missing values, clean column names, and filter by year
df.dropna(inplace=True)
df = columns_treatment(df)
df = df[df['YEAR'] > 1960]

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='DAILY_SUPPLY_OF_CALORIES', conn=conn)

df.head()

df.to_csv('../data/processed/daily_supply_of_calories.csv', index=False)

COUNTRY_NAME             object
COUNTRY_CODE             object
YEAR                      int64
DAILY_CALORIC_SUPPLY    float64
OVERWEIGHT_OR_OBESE     float64
dtype: object


In [ ]:
# Loading the data Access to Electricity
df = pd.read_csv('dataset\electry.csv')

# melting the table
df = pd.melt(df.drop(['Indicator Code','Indicator Name'],axis=1), id_vars=['Country Name','Country Code'], value_vars=None, var_name='YEAR', value_name='access_electricity', col_level=None, ignore_index=True)

# making all the columns names with Upper case
columns_treatment(df)

#dropping nan values
df.dropna(inplace=True)

# changing the type of columns 
df = df.astype({"YEAR": int})

print(df.dtypes)
df.head(100)

create_table(df=df,tablename='ELECTRICITY',conn=conn )
df

COUNTRY_NAME           object
COUNTRY_CODE           object
YEAR                    int32
ACCESS_ELECTRICITY    float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,ACCESS_ELECTRICITY
7980,Aruba,ABW,1990,100.000000
7985,Albania,ALB,1990,100.000000
7986,Andorra,AND,1990,100.000000
7988,United Arab Emirates,ARE,1990,100.000000
7989,Argentina,ARG,1990,92.154800
...,...,...,...,...
15955,Kosovo,XKX,2019,100.000000
15956,"Yemen, Rep.",YEM,2019,72.751701
15957,South Africa,ZAF,2019,85.000000
15958,Zambia,ZMB,2019,43.000000


In [22]:
# Load and clean obesity data
df = pd.read_csv('../data/raw/obesity.csv')

# Drop first column and rename for consistency
df = df.iloc[:, 1:]
df.rename(columns={
    'Country': 'Country name',
    'Obesity (%)': 'obesity_percentage'
}, inplace=True)

# Standardize column names
df = columns_treatment(df)

# Extract numeric value from obesity range string
df['OBESITY_PERCENTAGE'] = df['OBESITY_PERCENTAGE'].str.split(' ').str[0]

# Drop invalid or missing values
df.dropna(inplace=True)
df = df[df['OBESITY_PERCENTAGE'] != 'No']
df = df.astype({'OBESITY_PERCENTAGE': float})

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='OBESITY_PERCENTAGE', conn=conn)

df.head()


df.to_csv('../data/processed/obesity_percentage.csv', index=False)

COUNTRY_NAME           object
YEAR                    int64
OBESITY_PERCENTAGE    float64
SEX                    object
dtype: object


In [23]:
# Load and reshape population data
df = pd.read_csv('../data/raw/population.csv')

# Drop metadata columns and unpivot year columns
df = pd.melt(df.drop(['Indicator Code', 'Indicator Name'], axis=1),
             id_vars=['Country Name', 'Country Code'],
             var_name='YEAR', value_name='total_population')

# Standardize column names
df = columns_treatment(df)

# Clean missing values and fix data types
df.dropna(inplace=True)
df = df.astype({"YEAR": int})

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='POPULATION', conn=conn)

df.head(100)

df.to_csv('../data/processed/population.csv', index=False)

COUNTRY_NAME         object
COUNTRY_CODE         object
YEAR                  int32
TOTAL_POPULATION    float64
dtype: object


In [24]:
# Load and reshape rural population data
df = pd.read_csv('../data/raw/rural.csv')

# Drop metadata columns and unpivot year columns
df = pd.melt(df.drop(['Indicator Code', 'Indicator Name'], axis=1),
             id_vars=['Country Name', 'Country Code'],
             var_name='YEAR', value_name='rural_population')

# Standardize column names
df = columns_treatment(df)

# Clean missing values and fix data types
df.dropna(inplace=True)
df = df.astype({"YEAR": int})

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='RURAL', conn=conn)

df.head(100)

df.to_csv('../data/processed/rural.csv', index=False)

COUNTRY_NAME         object
COUNTRY_CODE         object
YEAR                  int32
RURAL_POPULATION    float64
dtype: object


In [25]:
# Load and reshape sanitation access data
df = pd.read_csv('../data/raw/sanitation.csv')

# Drop metadata columns and unpivot year columns
df = pd.melt(df.drop(['Indicator Code', 'Indicator Name'], axis=1),
             id_vars=['Country Name', 'Country Code'],
             var_name='YEAR', value_name='people_sanitation')

# Standardize column names
df = columns_treatment(df)

# Clean missing values and fix data types
df.dropna(inplace=True)
df = df.astype({"YEAR": int})

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='SANITATION', conn=conn)

df.head(100)


df.to_csv('../data/processed/sanitation.csv', index=False)

COUNTRY_NAME          object
COUNTRY_CODE          object
YEAR                   int32
PEOPLE_SANITATION    float64
dtype: object


In [26]:
# Load and filter industrialization intensity index data
df = pd.read_csv('../data/raw/Industrialization_intensity_index.csv')

# Keep only rows with numeric values
df = df[df['Subindicator Type'] == 'Value']
df.rename(columns={'Country ISO3': 'Country Code'}, inplace=True)

# Drop metadata and unpivot year columns
df = pd.melt(df.drop(['Indicator Id', 'Subindicator Type'], axis=1),
             id_vars=['Country Name', 'Country Code', 'Indicator'],
             var_name='YEAR', value_name='industrial_score')

# Standardize column names
df = columns_treatment(df)

# Clean missing values and fix data types
df.dropna(inplace=True)
df = df.astype({"YEAR": int})

# Keep only relevant indicator
df = df[df['INDICATOR'] == 'Competitive Industrial Performance Score']
df.drop('INDICATOR', axis=1, inplace=True)

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='INDUSTRIALIZATION', conn=conn)

df.head(100)

df.to_csv('../data/processed/Industrialization.csv', index=False)


COUNTRY_NAME         object
COUNTRY_CODE         object
YEAR                  int32
INDUSTRIAL_SCORE    float64
dtype: object


In [27]:
# Load and clean deaths by cause data
df = pd.read_csv('../data/raw/deaths_by_cause.csv')

# Rename key columns
df.rename(columns={
    'Entity': 'Country name',
    'Code': 'Country Code',
    'Number of executions (Amnesty International)': 'Executions'
}, inplace=True)

# Rename cause columns (e.g., "Deaths - Meningitis - ...") to simpler labels
new_columns = []
for column in df.columns:
    try:
        new_column = column.split(' - ')[1]
    except IndexError:
        new_column = column
        if column == "Terrorism (deaths)":
            new_column = "Terrorism"
    new_columns.append(new_column)
df.columns = new_columns

# Drop rows with missing data and standardize column names
df.dropna(inplace=True)
df = columns_treatment(df)

# Filter by year and drop unrelated columns
df = df[df['YEAR'] > 1960]
df = df.drop('EXECUTIONS', axis=1)

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='DEATHS_BY_CAUSE', conn=conn)

df.head()

df.to_csv('../data/processed/deaths_by_cause.csv', index=False)

COUNTRY_NAME                                   object
COUNTRY_CODE                                   object
YEAR                                            int64
MENINGITIS                                    float64
NEOPLASMS                                     float64
FIRE__HEAT__AND_HOT_SUBSTANCES                float64
MALARIA                                       float64
DROWNING                                      float64
INTERPERSONAL_VIOLENCE                        float64
HIV_AIDS                                      float64
DRUG_USE_DISORDERS                            float64
TUBERCULOSIS                                  float64
ROAD_INJURIES                                 float64
MATERNAL_DISORDERS                            float64
LOWER_RESPIRATORY_INFECTIONS                  float64
NEONATAL_DISORDERS                            float64
ALCOHOL_USE_DISORDERS                         float64
EXPOSURE_TO_FORCES_OF_NATURE                  float64
DIARRHEAL_DISEASES          

In [28]:
# Load and clean multiple causes of death data
df = pd.read_csv('../data/raw/number_of_deaths_by_multiple_causes.csv')

# Rename ID columns
df.rename(columns={'Entity': 'Country name', 'Code': 'Country Code'}, inplace=True)

# Simplify column names for causes
new_columns = []
for column in df.columns:
    try:
        new_column = column.split(' - ')[2].split(': ')[1]
    except IndexError:
        new_column = column
    new_columns.append(new_column)
df.columns = new_columns

# Drop missing values and clean columns
df.dropna(inplace=True)
df = columns_treatment(df)

# Filter by year
df = df[df['YEAR'] > 1960]

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='MULTIPLE_CAUSE_DEATHS', conn=conn)

df.head()

df.to_csv('../data/processed/multiple_cause_deaths.csv', index=False)

COUNTRY_NAME                                 object
COUNTRY_CODE                                 object
YEAR                                          int64
OUTDOOR_AIR_POLLUTION                       float64
HIGH_SYSTOLIC_BLOOD_PRESSURE                float64
DIET_HIGH_IN_SODIUM                         float64
DIET_LOW_IN_WHOLE_GRAINS                    float64
ALCOHOL_USE                                 float64
DIET_LOW_IN_FRUITS                          float64
UNSAFE_WATER_SOURCE                         float64
SECONDHAND_SMOKE                            float64
LOW_BIRTH_WEIGHT                            float64
CHILD_WASTING                               float64
UNSAFE_SEX                                  float64
DIET_LOW_IN_NUTS_AND_SEEDS                  float64
HOUSEHOLD_AIR_POLLUTION_FROM_SOLID_FUELS    float64
DIET_LOW_IN_VEGETABLES                      float64
LOW_PHYSICAL_ACTIVITY                       float64
SMOKING                                     float64
HIGH_FASTING

In [29]:
# Load and clean substance-related death data
df = pd.read_csv('../data/raw/substances-risk-factor-vs-direct-deaths.csv')

# Rename selected columns for clarity
df.rename(columns={
    'Entity': 'Country name',
    'Code': 'Country Code',
    'Deaths - Drug use disorders - Sex: Both - Age: All Ages (Number)': 'Drug use disorders',
    'Deaths - Alcohol use disorders - Sex: Both - Age: All Ages (Number)': 'Alcohol use disorders',
    'Deaths - Cause: All causes - Risk: Tobacco - Sex: Both - Age: All Ages (Number)': 'Tobacco'
}, inplace=True)

# Drop unused columns
df.drop([
    'Deaths - Cause: All causes - Risk: Drug use - Sex: Both - Age: All Ages (Number)',
    'Deaths - Cause: All causes - Risk: Alcohol use - Sex: Both - Age: All Ages (Number)'
], axis=1, inplace=True)

# Clean column names and filter by year
df.dropna(inplace=True)
df = columns_treatment(df)
df = df[df['YEAR'] > 1960]

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='SUBSTANCES_RISKFACTOR', conn=conn)

df.head()

df.to_csv('../data/processed/substances_riskfactor.csv', index=False)

COUNTRY_NAME              object
COUNTRY_CODE              object
YEAR                       int64
DRUG_USE_DISORDERS       float64
ALCOHOL_USE_DISORDERS    float64
TOBACCO                  float64
dtype: object


In [30]:
# Load and clean suicide rate data
df = pd.read_csv('../data/raw/suicide_rate.csv')

# Rename columns for consistency
df.rename(columns={
    'Location': 'Country Name',
    'SpatialDimValueCode': 'Country Code',
    'ParentLocationCode': 'Region Code',
    'ParentLocation': 'Region',
    'Period': 'Year',
    'Dim1': 'Sex',
    'Value': 'Suicides Range',
    'FactValueNumeric': 'Suicides',
    'FactValueNumericLow': 'SuicidesMin',
    'FactValueNumericHigh': 'SuicidesMax'
}, inplace=True)

# Drop unused metadata columns
df.drop([
    'IndicatorCode', 'Indicator', 'ValueType', 'Location type',
    'Period type', 'IsLatestYear', 'Dim1 type', 'Dim1ValueCode',
    'Language', 'DateModified'
], axis=1, inplace=True)

# Drop rows/columns with missing data and clean columns
df.dropna(axis=1, how='all', inplace=True)
df.dropna(inplace=True)
df = columns_treatment(df)

# Filter by year
df = df[df['YEAR'] > 1960]

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='SUICIDE_RATE', conn=conn)

df.head()

df.to_csv('../data/processed/suicide_rate.csv', index=False)

REGION_CODE        object
REGION             object
COUNTRY_CODE       object
COUNTRY_NAME       object
YEAR                int64
SEX                object
SUICIDES          float64
SUICIDESMIN       float64
SUICIDESMAX       float64
SUICIDES_RANGE     object
dtype: object


In [81]:
# Load and clean under-five mortality data
df = pd.read_csv('../data/raw/number_of_under_five_deaths.csv')

# Rename key columns
df.rename(columns={
    'Location': 'Country Name',
    'SpatialDimValueCode': 'Country Code',
    'ParentLocationCode': 'Region Code',
    'ParentLocation': 'Region',
    'Period': 'Year',
    'Dim1': 'Sex',
    'Value': 'u5 Deaths Range',
    'FactValueNumeric': 'u5 Deaths',
    'FactValueNumericLow': 'u5DeathsMin',
    'FactValueNumericHigh': 'u5DeathsMax'
}, inplace=True)

# Drop unused metadata columns
df.drop([
    'IndicatorCode', 'Indicator', 'ValueType', 'Location type',
    'Period type', 'IsLatestYear', 'Dim1 type', 'Dim1ValueCode',
    'Language', 'DateModified'
], axis=1, inplace=True)

# Drop missing data and clean column names
df.dropna(axis=1, how='all', inplace=True)
df.dropna(inplace=True)
df = columns_treatment(df)

# Filter by year and drop unnecessary detail columns
df = df[df['YEAR'] > 1960]
df.drop(['REGION', 'REGION_CODE', 'U5DEATHSMIN', 'U5DEATHSMAX', 'U5_DEATHS_RANGE'], axis=1, inplace=True)

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='UNDER5_DEATHS', conn=conn)

df.head()


df.to_csv('../data/processed/under5_deaths.csv', index=False)

COUNTRY_CODE    object
COUNTRY_NAME    object
YEAR             int64
SEX             object
U5_DEATHS        int64
dtype: object


In [82]:
# Load and clean homicide estimate data
df = pd.read_csv('../data/raw/WHO_estimatesofhomicides.csv')

# Rename key columns
df.rename(columns={
    'Location': 'Country Name',
    'SpatialDimValueCode': 'Country Code',
    'ParentLocationCode': 'Region Code',
    'ParentLocation': 'Region',
    'Period': 'Year',
    'Dim1': 'Sex',
    'Value': 'Homicides Range',
    'FactValueNumeric': 'Homicides estimates',
    'FactValueNumericLow': 'homicidesMin',
    'FactValueNumericHigh': 'homicidesMax'
}, inplace=True)

# Drop unused metadata columns
df.drop([
    'IndicatorCode', 'Indicator', 'ValueType', 'Location type',
    'Period type', 'IsLatestYear', 'Dim1 type', 'Dim1ValueCode',
    'Language', 'DateModified'
], axis=1, inplace=True)

# Drop missing values and clean column names
df.dropna(axis=1, how='all', inplace=True)
df.dropna(inplace=True)
df = columns_treatment(df)

# Filter by year and drop unnecessary columns
df = df[df['YEAR'] > 1960]
df.drop([
    'HOMICIDESMIN', 'HOMICIDESMAX', 'HOMICIDES_RANGE',
    'REGION_CODE', 'REGION'
], axis=1, inplace=True)

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='HOMICIDES_ESTIMATES', conn=conn)

df.head()



df.to_csv('../data/processed/homicides_estimates.csv', index=False)

COUNTRY_CODE            object
COUNTRY_NAME            object
YEAR                     int64
SEX                     object
HOMICIDES_ESTIMATES    float64
dtype: object


In [89]:
# Load and merge BMI data for males and females
dfmales = pd.read_csv('../data/raw/Mean-body-mass-index-bmi-in-adult-males.csv')
dfwomen = pd.read_csv('../data/raw/Mean-body-mass-index-bmi-in-adult-women.csv')

# Merge on country and year
df = pd.merge(dfmales, dfwomen, on=['Entity', 'Code', 'Year'])

# Rename columns for consistency
df.rename(columns={
    'Entity': 'Country Name',
    'Code': 'Country Code',
    'Mean BMI (male)': 'bmi_male',
    'Mean BMI (female)': 'bmi_female'
}, inplace=True)

# Drop missing values and clean column names
df.dropna(axis=1, how='all', inplace=True)
df.dropna(inplace=True)
df = columns_treatment(df)

# Filter by year
df = df[df['YEAR'] > 1960]

# Check data types
print(df.dtypes)

# Save to CSV
df.to_csv('../data/processed/mean_bmi.csv', index=False)

# Save to SQL (optional)
# create_table(df=df, tablename='MEAN_BMI', conn=conn)

# Preview
df.head()


COUNTRY_NAME     object
COUNTRY_CODE     object
YEAR              int64
BMI_MALE        float64
BMI_FEMALE      float64
dtype: object


,COUNTRY_NAME,COUNTRY_CODE,YEAR,BMI_MALE,BMI_FEMALE
0,Afghanistan,AFG,1975,18.999440,18.844326
1,Afghanistan,AFG,1976,19.105518,18.986417
2,Afghanistan,AFG,1977,19.212080,19.128086
3,Afghanistan,AFG,1978,19.319053,19.269247
4,Afghanistan,AFG,1979,19.425952,19.409477


In [83]:
# Load and reshape air pollution exposure data
df = pd.read_csv('../data/raw/Pastas/theworldbank_airpollution/API_EN.ATM.PM25.MC.ZS_DS2_en_csv_v2_3481084.csv')

# Drop metadata columns and unpivot year columns
df = pd.melt(df.drop(['Indicator Code', 'Indicator Name'], axis=1),
             id_vars=['Country Name', 'Country Code'],
             var_name='YEAR', value_name='Percentage of People Exposed to Air Pollution')

# Clean missing values and column names
df.dropna(inplace=True)
df = columns_treatment(df)

# Convert year column to integer
df = df.astype({"YEAR": int})

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='AIRPOLLUTION', conn=conn)

df.head()


df.to_csv('../data/processed/airpollution.csv', index=False)

COUNTRY_NAME                                      object
COUNTRY_CODE                                      object
YEAR                                               int32
PERCENTAGE_OF_PEOPLE_EXPOSED_TO_AIR_POLLUTION    float64
dtype: object


In [84]:
# Load and reshape CO2 emissions data
df = pd.read_csv('../data/raw/Pastas/theworldbank_co2emissions/API_EN.ATM.CO2E.KT_DS2_en_csv_v2_3470002.csv')

# Drop metadata columns and unpivot year columns
df = pd.melt(df.drop(['Indicator Code', 'Indicator Name'], axis=1),
             id_vars=['Country Name', 'Country Code'],
             var_name='YEAR', value_name='CO2 emissions')

# Clean missing values and column names
df.dropna(inplace=True)
df = columns_treatment(df)

# Convert year column to integer
df = df.astype({"YEAR": int})

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='CO2EMISSIONS', conn=conn)

df.head()

df.to_csv('../data/processed/co2emissions.csv', index=False)

COUNTRY_NAME      object
COUNTRY_CODE      object
YEAR               int32
CO2_EMISSIONS    float64
dtype: object


In [85]:
# Load and reshape CPIA corruption index data
df = pd.read_csv('../data/raw/Pastas/theworldbank_transparencycorruption/API_IQ.CPA.TRAN.XQ_DS2_en_csv_v2_3469422.csv')

# Drop metadata columns and unpivot year columns
df = pd.melt(df.drop(['Indicator Code', 'Indicator Name'], axis=1),
             id_vars=['Country Name', 'Country Code'],
             var_name='YEAR', value_name='CPIA corruption in the public sector from 1 to 6')

# Clean missing values and column names
df.dropna(inplace=True)
df = columns_treatment(df)

# Convert year column to integer
df = df.astype({"YEAR": int})

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='CPIA_CORRUPTION', conn=conn)

df.head()

df.to_csv('../data/processed/cpia_corruption.csv', index=False)

COUNTRY_NAME                                         object
COUNTRY_CODE                                         object
YEAR                                                  int32
CPIA_CORRUPTION_IN_THE_PUBLIC_SECTOR_FROM_1_TO_6    float64
dtype: object


In [86]:
# Load and reshape urban population data
df = pd.read_csv('../data/raw/Pastas/theworldbank_urbanpopulation/API_SP.URB.TOTL_DS2_en_csv_v2_3477498.csv')

# Drop metadata columns and unpivot year columns
df = pd.melt(df.drop(['Indicator Code', 'Indicator Name'], axis=1),
             id_vars=['Country Name', 'Country Code'],
             var_name='YEAR', value_name='Urban Population')

# Clean missing values and column names
df.dropna(inplace=True)
df = columns_treatment(df)

# Convert year column to integer
df = df.astype({"YEAR": int})

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='URBAN_POPULATION', conn=conn)

df.head()

df.to_csv('../data/processed/urban_population.csv', index=False)

COUNTRY_NAME         object
COUNTRY_CODE         object
YEAR                  int32
URBAN_POPULATION    float64
dtype: object


In [87]:
# Load and reshape adult literacy rate data
df = pd.read_csv('../data/raw/Literacy rate, adult total (% of people ages 15 and above ).csv')

# Drop metadata columns and unpivot year columns
df = pd.melt(df.drop(['Indicator Code', 'Indicator Name'], axis=1),
             id_vars=['Country Name', 'Country Code'],
             var_name='YEAR', value_name='Literacy Rate')

# Clean missing values and column names
df.dropna(inplace=True)
df = columns_treatment(df)

# Convert year column to integer
df = df.astype({"YEAR": int})

print(df.dtypes)

# Save to SQL
# create_table(df=df, tablename='LITERACY_RATE', conn=conn)

df.head()

df.to_csv('../data/processed/literacy_rate.csv', index=False)

COUNTRY_NAME      object
COUNTRY_CODE      object
YEAR               int32
LITERACY_RATE    float64
dtype: object


In [93]:
# Create the base from life expectancy
# df = pd.read_sql("SELECT * FROM LIFE_EXPECTANCY;", conn)

# Merge all datasets step by step using SQL and the original `mg()` function
# df_final = mg("SELECT * FROM ADULT_MORTALITY;", conn, df)
# df_final = mg("SELECT * FROM airpollution;", conn, df_final)
# df_final = mg("SELECT * FROM co2emissions;", conn, df_final)
# df_final = mg("SELECT * FROM cpia_corruption;", conn, df_final)
# df_final = mg("SELECT * FROM daily_supply_of_calories;", conn, df_final)
# df_final = mg("SELECT * FROM deaths_by_cause;", conn, df_final)
# df_final = mg("SELECT * FROM electricity;", conn, df_final)
# df_final = mg("SELECT * FROM gdp;", conn, df_final)
# df_final = mg("SELECT * FROM homicides_estimates;", conn, df_final)
# df_final = mg("SELECT * FROM literacy_rate;", conn, df_final)
# df_final = mg("SELECT * FROM mean_bmi;", conn, df_final)
# df_final = mg("SELECT * FROM overweight_in_children;", conn, df_final)
# df_final = mg("SELECT * FROM population;", conn, df_final)
# df_final = mg("SELECT * FROM rural;", conn, df_final)
# df_final = mg("SELECT * FROM sanitation;", conn, df_final)
# df_final = mg("SELECT * FROM substances_riskfactor;", conn, df_final)
# df_final = mg("SELECT * FROM suicide_rate;", conn, df_final)
# df_final = mg("SELECT * FROM under5_deaths;", conn, df_final)
# df_final = mg("SELECT * FROM urban_population;", conn, df_final)
# df_final = mg("SELECT * FROM obesity_percentage;", conn, df_final)

# Drop unnecessary columns if they exist
# df_final = df_final.drop(['suicides_range', 'region', 'region_code'], axis=1, errors='ignore')

# Save final merged table to SQL
# create_table(df=df_final, tablename='FINAL_TABLE', conn=conn)

# Preview the result
# df_final.head()


In [92]:
# Load the base life expectancy dataset
df = pd.read_csv('../data/processed/Life_expectancy.csv')

# Merge each dataset into df using mg_csv()
df_final = mg_csv(pd.read_csv('../data/processed/Adult_mortality_rate.csv'), df)
df_final = mg_csv(pd.read_csv('../data/processed/airpollution.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/co2emissions.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/cpia_corruption.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/daily_supply_of_calories.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/deaths_by_cause.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/gdp.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/homicides_estimates.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/literacy_rate.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/mean_bmi.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/overweight_in_children.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/population.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/rural.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/sanitation.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/substances_riskfactor.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/suicide_rate.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/under5_deaths.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/urban_population.csv'), df_final)
df_final = mg_csv(pd.read_csv('../data/processed/obesity_percentage.csv'), df_final)

# Drop unnecessary columns if they exist
df_final = df_final.drop(columns=['SUICIDES_RANGE', 'REGION', 'REGION_CODE'], errors='ignore')

# Save final merged dataset to CSV
df_final.to_csv('../data/processed/final_table.csv', index=False)

# Preview the result
df_final.head()

,COUNTRY_NAME,COUNTRY_CODE,YEAR,LIFE_EXPECTANCY,ADULT_MORTALITY_RATE_PROBABILITY_OF_DYING_BETWEEN_15_AND_60_YEARS_PER_1000_POPULATION,PERCENTAGE_OF_PEOPLE_EXPOSED_TO_AIR_POLLUTION,CO2_EMISSIONS,CPIA_CORRUPTION_IN_THE_PUBLIC_SECTOR_FROM_1_TO_6,DAILY_CALORIC_SUPPLY,OVERWEIGHT_OR_OBESE,...,PEOPLE_SANITATION,DRUG_USE_DISORDERS_y,ALCOHOL_USE_DISORDERS_y,TOBACCO,SUICIDES,SUICIDESMIN,SUICIDESMAX,U5_DEATHS,URBAN_POPULATION,OBESITY_PERCENTAGE
0,Aruba,ABW,1960,65.662000,NaN,NaN,11092.6750,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27525.0,NaN
1,Africa Eastern and Southern,AFE,1960,42.716053,NaN,NaN,118545.9013,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19239140.0,NaN
2,Afghanistan,AFG,1960,32.446000,NaN,NaN,414.3710,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,755835.0,NaN
3,Africa Western and Central,AFW,1960,37.205380,NaN,NaN,8760.4630,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14141671.0,NaN
4,Angola,AGO,1960,37.524000,NaN,NaN,550.0500,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,569223.0,NaN


In [95]:
# Filter the final dataset to only include Brazil
df_final = df_final[df_final['COUNTRY_NAME'] == 'Brazil']

# Inspect column names
df_final.columns

# Check data types
df_final.dtypes

# Close SQL connection (if it was used)
# conn.close()


COUNTRY_NAME                                                                              object
COUNTRY_CODE                                                                              object
YEAR                                                                                       int64
LIFE_EXPECTANCY                                                                          float64
ADULT_MORTALITY_RATE_PROBABILITY_OF_DYING_BETWEEN_15_AND_60_YEARS_PER_1000_POPULATION    float64
PERCENTAGE_OF_PEOPLE_EXPOSED_TO_AIR_POLLUTION                                            float64
CO2_EMISSIONS                                                                            float64
CPIA_CORRUPTION_IN_THE_PUBLIC_SECTOR_FROM_1_TO_6                                         float64
DAILY_CALORIC_SUPPLY                                                                     float64
OVERWEIGHT_OR_OBESE                                                                      float64
MENINGITIS                    